In [36]:
import codecs
import xlrd
import numpy as np
import jieba.posseg as psg # 分词
import jieba.analyse as analyse # 关键词提取
import gensim
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
from wordcloud import WordCloud

In [37]:
#读取语料
def get_excelRC(dir, sheetIndex=0):
    """
    读取文本的行列
    """
    data = xlrd.open_workbook(dir)
    table = data.sheets()[sheetIndex]
    nrows = table.nrows # 行数
    ncols = table.ncols # 列数
    return table, nrows, ncols

def get_frame():
    """
    读取文本,返回内容列表及时间代码列表
    BG.xlsx文件一共有5列数据
    
    """    
    dir_frame = "./data/BG.xlsx"
    table, nrows, ncols = get_excelRC(dir_frame)
    start_row = 1
    end_row = nrows
    list_ctHandle = []
    list_stkcd = []
    for x in range(start_row, end_row):
        row = table.row_values(x)
        if len(row[4]) != 0: # 去掉没有内容的部分
            list_stkcd.append(str(row[1]) + '_' + str(row[2])) # 唯一值，相当于索引
            list_ctHandle.append(row[4]) # 文本内容列表
    return list_ctHandle,list_stkcd   

In [38]:
list_ctHandle,list_stkcd = get_frame()
print(list_ctHandle[:1])
print(list_stkcd)

['1、自然因素：自然资源的特点和在人类生产与生活中的地位和作用，决定了一个国家所拥有的自然资源的数量、质量和构成及其分布状况，对经济发展关系极大。2、人口数量：人是生产者与消费者的统一，人口发展必须与物质资料生产发展相适应。一方面，人作为生产者，是社会生产力的主体，一定的人口数量是经济发展所不可缺少的，适当的人口增长也是推动经济发展的一个因素。另一方面，人作为生产者又是有条件的，不仅要同生产资料相结合，还要受年龄、体质和技能的限制。3、科学技术：科学技术作为第一生产力，不仅是经济增长的决定性因素，对经济发展的其他方面也起着巨大的推动作用。4、文化教育：对人力资源的开发是通过人力投资实现的，它包括两个方面，一是体力投资，一是智力投资。教育是对人的智力投资，是提高人力资源质量的各种途径中最重要的途径。']
['1000001.0_2016.0', '1000002.0_2016.0', '1000003.0_2016.0', '1000004.0_2016.0', '1000005.0_2016.0']


In [39]:
stopwords = [w.strip() for w in codecs.open('./data/chineseStopWords.txt', 'r', encoding='utf-8')]
print(stopwords)

['?', '、', '。', '“', '”', '《', '》', '!', '！', '，', ',', '：', ':', '；', '？', '-', '(', ')', '（', '）', '·', '--', '……', '/', '.', '|', '——', '‘', '’', '□', '【', '】', 'A', 'B', 'C', 'D', '啊', '阿', '哎', '哎呀', '哎哟', '唉', '俺', '俺们', '按', '按照', '吧', '吧哒', '把', '罢了', '被', '本', '本着', '比', '比方', '比如', '鄙人', '彼', '彼此', '边', '别', '别的', '别说', '并', '并且', '不比', '不成', '不单', '不但', '不独', '不管', '不光', '不过', '不仅', '不拘', '不论', '不怕', '不然', '不如', '不特', '不惟', '不问', '不只', '朝', '朝着', '趁', '趁着', '乘', '冲', '除', '除此之外', '除非', '除了', '此', '此间', '此外', '从', '从而', '出', '打', '待', '但', '但是', '当', '当着', '到', '得', '的', '的话', '等', '等等', '地', '第', '叮咚', '对', '对于', '多', '多少', '而', '而况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '刚', '刚刚', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或者', '极了', '及', '及其', '及

In [40]:
# 分词
def is_chinese(uchar):
    if uchar > u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False
    
def cut_words(eachTxt, n=1):
    """
    jieba切词只保留中文词语，n:保留词的长度，返回分词列表
    """
    words = psg.cut(eachTxt)
    words_list = []
    for w in words:
        flag = True
        for i in range(len(w.word)):
            if not is_chinese(w.word[i]) or w.word in stopwords:
                flag = False
                break
        if flag and len(w.word) > n:
            words_list.append(w.word)
    return words_list


In [42]:
segs = []
for i in range(len(list_ctHandle)):
    terms = cut_words(list_ctHandle[i]) 
    terms.append('\n')
    segs.append(" ".join(terms))
print(segs[:1])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wei.li\AppData\Local\Temp\jieba.cache
Loading model cost 0.845 seconds.
Prefix dict has been built successfully.


['自然 因素 自然资源 人类 生产 生活 地位 作用 国家 拥有 自然资源 数量 质量 分布 状况 经济 发展 关系 极大 人口数量 生产者 消费者 人口 发展 物质 资料 生产 发展 相适应 生产者 社会 生产力 主体 人口数量 经济 发展 缺少 人口 增长 推动 经济 发展 因素 生产者 条件 生产资料 相结合 还要 年龄 体质 技能 科学技术 科学技术 生产力 经济 增长 决定性 因素 经济 发展 推动 作用 文化教育 人力资源 开发 人力 投资 包括 两个 体力 投资 智力投资 教育 智力投资 提高 人力资源 质量 途径 途径 \n']


In [46]:
# TF-IDF
content = ''.join(segs)
tfidf_words = analyse.extract_tags(content, topK=10, withWeight=True)
print("【TF-IDF】")
print(tfidf_words)

# TextRank
content = ''.join(segs)
tr_words = analyse.textrank(content, topK=10, withWeight=True)
print("【TextRank】")
print(tr_words)


【TF-IDF】
[('全球化', 0.2917480270424312), ('经济', 0.24183097352646793), ('发展', 0.1393947187098165), ('生产', 0.08076386665137615), ('世界', 0.0801381542306422), ('跨国', 0.07860512714412844), ('劳动力', 0.07657548133497707), ('国家', 0.07340266774605504), ('全球', 0.06745935486729357), ('各国', 0.06552358468612385)]
【TextRank】
[('经济', 1.0), ('发展', 0.6230042173291817), ('全球化', 0.49302642592979634), ('生产', 0.30625172645429427), ('世界', 0.294605316937291), ('国家', 0.2686683099468205), ('规模', 0.19162753760027842), ('全球', 0.1860374808984976), ('贸易', 0.1770559747283987), ('科技', 0.17024958240394938)]


In [ ]:
# word2vec
model = gensim.models.word2vec.Word2vec.load('./model/word2vec')
def predict_proba(oword, iword):
    """
    计算某词对于模型中各个词的转义概率p(wk|wi)
    """
    iword_vec = model[iword] # 获取输入词的词向量
    oword = model.wv.vocab[oword] # 获取保存权重的词的词库
    oword_l = model.trainables.synl[oword.point].T
    dot = np.dot(iword_vec, oword_l)
    lprob = -sum(np.logaddexp(0,-dot) + oword.code * dot)
    return lprob

def keywords(s):
    """
    各个词对于某词wi转移概率的乘积即为p(content|wi),
    如果p(content|wi)越大说明在出现wi这个词的条件下，此内容概率越大，
    那么把所有词的p(content|wi)按照大小相许排列，越靠前的词越重要，越应该看成文本的关键词
    """
    s = [w for w in s if w in model] # 抽出s中与训练model的词
    ws = {w:sum([predict_proba(u, w) for u in s]) for w in s}
    return Counter(ws).most_common()
    
    
    
    
    
    
    
    
    
    